<a href="https://colab.research.google.com/github/harmanbirstudy/LLMsPractice/blob/main/gemini2_0_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=vxF8-ay9Bzk

https://github.com/GoogleCloudPlatform/generative-ai -- good examples

https://github.com/GoogleCloudPlatform/genai-product-catalog/tree/main

1) Generic run for gemini 2.0 , key from cola secrets

In [ ]:
pip install google-genai

In [ ]:
from google import genai
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))
response = client.models.generate_content(model='gemini-2.0-flash-exp', contents='How does AI work?')
print(response.text)

2) Langchain with gemini

https://python.langchain.com/docs/integrations/llms/google_ai/

https://python.langchain.com/docs/concepts/chat_models/

In [ ]:
!pip install --quiet langchain

In [ ]:
!pip install --quiet langchain-google-genai

In [2]:
from langchain_google_genai import GoogleGenerativeAI
from google.colab import userdata
api_key=userdata.get('GEMINI_API_KEY')

In [3]:
model = GoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=api_key)

In [ ]:
question1 = "What is BMW revenue for 2022 break it up per line of business and share the 2024 business prespective? "

# for chunk in model.stream(question1):
#   print(chunk, end="")

print(model.invoke(question1))

Using prompts

In [ ]:
from langchain_core.prompts import PromptTemplate
template = "What is {company} revenue for 2022 break it up per line of business and share the {year} business prespective? "

prompt = PromptTemplate.from_template(template)

chain = prompt | model

# for chunck in chain.stream({"company": "BMW", "year": "2024"}):
#   print(chunck, end="")

print(chain.invoke({"company": "BMW", "year": "2024"}))


More params like limit response and print config etc

In [ ]:
for chunk in model.invoke(question1, stream=True, stop=["growth"], temperature=0.2):
  print(chunk, end="" , flush=True)

In [ ]:
response = model.generate([question1])
#response.generations[0][0].text
response.generations[0][0].generation_info

3) Langchain gemini ChatModel

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
chat_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", google_api_key=api_key)

anwser = chat_model.invoke([HumanMessage(content=question1)])
print(anwser.content)

4)Gemini Image models

gemini-pro-vision -- depreciated so using gemini-2.0 unless found a better one

In [ ]:
anwser = chat_model.invoke([HumanMessage(content="Hello. Who are you")])
print(anwser.content)


In [ ]:
from langchain_google_genai._image_utils import ImageBytesLoader
image_loader = ImageBytesLoader()
image = image_loader.load_bytes("https://images.pexels.com/photos/8827237/pexels-photo-8827237.jpeg")
from IPython.display import Image , display
display(Image(image))

In [ ]:
type(image)

In [ ]:
text_message = {
    "type":"text",
    "text": "Describe the image"
}
image_message = {
    "type":"image_url",
    "image_url": {"url":"https://images.pexels.com/photos/8827237/pexels-photo-8827237.jpeg"}
}

message = HumanMessage(content=[text_message, image_message])

anwser = chat_model.invoke([message])
print(anwser.content)

In [ ]:
image_url_nyse =(
    "https://www.reuters.com/resizer/v2/https%3A%2F%2Fcloudfront-us-east-2.images.arcpublishing.com%2Freuters%2FYSJL2MEQYJPPJFTB64XXCPG2FA.jpg?auth=b81bf08306817072ac6e4e3b9f0419a73895be15cf490bb7bfad62ec67bc1c5e&width=4996&quality=80"
)

image_message = {
    "type":"image_url",
    "image_url": {"url":image_url_nyse}
}

prompt = "Describe the scene , be detailed what is happening"

message = HumanMessage(content=[prompt, image_message])

output = chat_model.invoke([message])
print(output.content)
display(Image(url=image_url_nyse))

Using Prompts to hint what we expect it to think.City/Landmarks

In [ ]:
prompt1 = """{"city":"London" ,"Landmark":"Big ben" , "Postivie Historical event:The Great Fire of 1666 while devastating, it ultimately led to a major rebuilding effort that resulted in a new, more planned and improved London, essentially giving birth to a modern version of the city}"""

prompt2 = """{"city":"Paris" ,"Landmark":"Eiffel Tower" , "Postivie Historical event:Founded in 1257 The soborne university becamme a a major center of learning in Europe and the core of the University of Paris }"""


image_url1 = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark1.jpg"
image_url2 = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark2.jpg"
image_url3 = "https://storage.googleapis.com/github-repo/img/gemini/intro/landmark3.jpg"

image_message1 = {
    "type":"image_url",
    "image_url": {"url":image_url1}
}

image_message2 = {
    "type":"image_url",
    "image_url": {"url":image_url2}
}

image_message3 = {
    "type":"image_url",
    "image_url": {"url":image_url3}
}

message1 = HumanMessage(content=[image_message1, prompt1,image_message2, prompt2,image_message3])

output = chat_model.invoke([message1])

print(prompt1)
display(Image(url=image_url1))
print(prompt2)
display(Image(url=image_url2))

print(output.content)
display(Image(url=image_url3))


Same above example but with stocks

In [ ]:
prompt1 = """{"Ticker":"GOOG" ,"Industry":"Technology" , "Description:Alphabet stock is up by 8.07% in past 6M. It is trading at $133.84 with an increase of 0.12% after hours.The stock prise had risen compared to Nov2023}"""

prompt2 = """{"Ticker":"AMZN" ,"Industry":"E-commerce" , "Description:Amazon stock is up by 2.63% in past 1M. It is trading at $149.97 with an no change of 0.00% after hours.The stock prise had risen compared to Nov 27}"""


image_url1 = "https://storage.googleapis.com/oss-llm-bucket-public/stock-1.png"
image_url2 = "https://storage.googleapis.com/oss-llm-bucket-public/stock-2.png"
image_url3 = "https://storage.googleapis.com/oss-llm-bucket-public/stock-3.png"

image_message1 = {
    "type":"image_url",
    "image_url": {"url":image_url1}
}

image_message2 = {
    "type":"image_url",
    "image_url": {"url":image_url2}
}

image_message3 = {
    "type":"image_url",
    "image_url": {"url":image_url3}
}

message1 = HumanMessage(content=[image_message1, prompt1,image_message2, prompt2,image_message3])

output = chat_model.invoke([message1])

print(prompt1)
display(Image(url=image_url1))
print(prompt2)
display(Image(url=image_url2))

print(output.content)
display(Image(url=image_url3))